In [ ]:
import warnings
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes, fetch_openml,load_iris,fetch_california_housing
from sklearn.feature_selection import mutual_info_regression, f_regression, RFE, SelectFromModel, SelectKBest, f_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import (
RepeatedStratifiedKFold, 
cross_val_score, 
train_test_split, 
GridSearchCV,
cross_val_predict, 
learning_curve, 
validation_curve)
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error,zero_one_loss, roc_auc_score,root_mean_squared_error
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.datasets import make_circles, make_moons, make_blobs
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from os.path import join as pjoin
from mlxtend.evaluate import bias_variance_decomp
from sklearn.feature_extraction.text import TfidfVectorizer
#sharper plots
%config InlineBackend.figure_format = 'retina'

from sklearn.linear_model import (LogisticRegression, LogisticRegressionCV,
                                  SGDClassifier)

warnings.filterwarnings("ignore")

# LIME
Рассмотрим метод LIME (Local Interpretable Model Agnostic explanations), который позволяет построить интерпретацию для некоторого объясняемого объекта x* и его окрестности.
Для этого независимо от модели $a(x)$ для объекта вводится его объясняемое представление, и обучается модель $\hat a(x)$, которая строит предсказания для этих признаков. Чаще всего для этого используются довольно простые представления (например, мешок слов или суперпиксели). 
Для построения объясняющей модели:
1) Строится суррогатная выборка $X_{x*}^l = {(\hat x_i, y_i)}$ :
    1) создается l примеров $\hat x_i$ путем пертурбации исходных признаков, например, обнуления случайного количества единиц в интерпретируемом представлении $\hat x*$ объекта для текстов или  картинок. Вопрос, как в примере с текстом проходит обнуление?
    3) для каждого $\hat x_i$ делается переход в исходное пространство 
    4) получается $y = a(x_i)$

2) Обучается объясняющая модель:
   $$a_i = argmin_{b \in B} {\sum_{x_i} {\pi_{x*}(x_i)(a(x_i)-b((\hat x_i))^2+\sigma(b)}},$$
    - $B$ - cемейство объясняющих моделей, $\sigma(b)$ - ее сложность
    - $\pi_{x*}(x_i)$ - вес объекта, полученный с помощью некоторого ядра
В качестве функции ошибки можно использовать и другую какую-то.
Пример - разреженные линейные представления (SLE), если использовать квадратичную функцию и $\sigma(b) = \infty[||w_b||_0 K]$ (Что за норма такая $||w_b||_0$?). Для интерпретации в таком случае достаточно вывести признаки с ненулевыми весами - так мы увидим, какие слова были использованы как самые "важные"

В итоге мы получаем довольно неплохую локальную интерпретацию, но глобально она будет, конечно же, не очень.

На практике обычно оптимизируется только часть, относящаяся к лоссу, а сложность контролируется за счет регуляризации объясняющей модели или RFE. 
Как это делается (для регуляризации): Задается большое $\lambda$, после чего оно постепенно уменьшается, пока не будет достигнуто K признаков с ненулевыми весами.

Как LIME работает с таблицами?
Во-первых, используются исходные признаки, не бинарные представления.
Во-вторых, выборки строятся путем вытягивания значения из нормального распределения со средним и стандартным отклонением, полуенными из обучающего набора для каждой фичи независимо.
Кажется, что это не поможет нам строить интерпретацию для конкретного семпла. Но это работает, по крайней мере, частично. Вопрос: почему?

## Таблицы

Для демонстрации используем уже знакомый нам датасет про дома

In [ ]:
data_path = r"C:\Users\nikol_ri8fhbe\Documents\ml" 

In [ ]:
data = pd.read_csv(pjoin(data_path, "realestate.txt"), sep="\t")
data.fillna(data.median(), inplace=True)
X = data.drop("SalePrice", axis=1)
y = data["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11)

In [ ]:
y.mean(), y.min(), y.max(), y.std()

In [ ]:
data.columns

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = # Обучите случайный лес 


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model.score(X_test, y_test)
root_mean_squared_error(y_pred, y_test)

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.values.tolist(),
                                                  class_names=['SalePrice'], verbose=True, mode='regression', discretize_continuous=True)

In [ ]:
j = 5
exp = explainer.explain_instance(X_test.values[j], model.predict, num_features=6)

In [ ]:
exp.show_in_notebook(show_table=True)

Здесь положительное влияние показано справа, отрицательное - слева. Значения соответствуют весам линейной модели, аппроксимирующей наш случайный лес. Грубо говоря, если мы уменьшим year на 12, то и наше предсказание уменьшится на 31

Проверим, так ли

In [ ]:
X_test.values[j]

In [ ]:
print('Original prediction:', model.predict(X_test.values[j].reshape(1, -1)))
tmp = X_test.values[j].copy()
tmp[6] = 1980

In [ ]:
print('Prediction removing some features:',  model.predict(tmp.reshape(1, -1)))
print('Difference:', model.predict(tmp.reshape(1, -1)) - model.predict(X_test.values[j].reshape(1, -1)))

In [ ]:
exp.show_in_notebook(show_table=True)

Интересно, что для категориальных фичей мы снова можем построить более удачные объяснения. Как мы поняли, LIME требует бинарные представления. Как мы можем к ним придти? 

In [ ]:
X_train.head()

Мы можем задать, какие фичи у нас категориальные. 

In [ ]:
categorical_features = ["Beds", "Baths", "Air", "Garage", "Pool", "Quality", "Style", "Highway"]
categorical_feature_ids = [1,2,3,4,5,7,8,10]

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train.values, 
    feature_names=X_train.columns.values.tolist(),
    class_names=['SalePrice'], 
    categorical_features=categorical_feature_ids,
    categorical_names=categorical_features,
    verbose=True,
    mode='regression', 
    discretize_continuous=True
)
# Объясните тот же пример

Можно вообще извратиться и сделать за LIME бинаризацию. 

In [ ]:
encoder = # Бинаризуйте выбранные фичи с помощью sklearn-совместимого энкодера. Названия можно получить через get_feature_names_out()
cat_df = # your code
X_train_ohe = pd.concat([X_train[["SqFeet","Year", "Lot"]], cat_df], axis=1)

In [ ]:
X_train_ohe.head()

In [ ]:
cat_df = # your code
X_test_ohe = pd.concat([X_test[["SqFeet","Year", "Lot"]], cat_df], axis=1)

In [ ]:
encoder.get_feature_names_out()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train_ohe, y_train)

In [ ]:
explainer = # Настройте explainer

In [ ]:
# Постройте интерпретацию для 5 семпла

Мы можем также задать ширину ядра и его вид.

**Задание**: Напишите свои ядра, берущие на вход расстояния и возвращающие значения от 0 до 1. Замените ядро по умолчанию. 

## Тексты

Также рассмотрим интерпретацию для классификации новостей на две группы - атеизм и христианство.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']

Векторизуем тексты с помощью TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = # сделайте то же самое и для теста

In [ ]:
rf = # Обучите случайный лес

In [ ]:
pred = rf.predict(test_vectors)
f1_score(newsgroups_test.target, pred, average='binary')

In [ ]:
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)

In [ ]:
print(c.predict_proba([newsgroups_test.data[0]]))

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
idx = 83
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

In [ ]:
exp.as_list()

In [ ]:
exp.show_in_notebook(text=False)

Посмотрим, сохраняется ли ситуация с изменением фичей. Тут ожидание следующее - вероятность увеличится на 0.27, если мы уберем из текста слова Host и Posting.

In [ ]:
print('Original prediction:', rf.predict_proba(test_vectors[idx])[0,1])
tmp = test_vectors[idx].copy()
tmp[0,vectorizer.vocabulary_['Posting']] = 0
tmp[0,vectorizer.vocabulary_['Host']] = 0
print('Prediction removing some features:', rf.predict_proba(tmp)[0,1])
print('Difference:', rf.predict_proba(tmp)[0,1] - rf.predict_proba(test_vectors[idx])[0,1])

Для текстов мы можем еще и посмотреть на самые важные слова:

In [ ]:
exp.show_in_notebook(text=True)

Судя по всему, на самом деле моделька смотрит на мусор типа заголовков или частых слов. Проверим, так ли это для какого-то другого примера:

In [ ]:
idx = 34
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
print('True class: %s' % class_names[newsgroups_test.target[idx]])
exp.show_in_notebook(text=True)

# Итого
Плюсы: 
- Можно поменять предсказывающую модель, но все еще использовать ту же объясняющую
- Легко применить, работает с разными видами данных
  
Минусы:
- Для табличных данных очень сложно выбрать ядро, да и вообще выбор ядра может значительно поменять интерпретацию.
- Сложность объяснения должна быть выбрана заранее
- Нестабильный

**Задание(*):**
У вас есть все составляющие LIME - ядра, алгоритм, кодировка фичей, loess с прошлой пары. Не хватает только кода с семплированием.
Напишите свой класс Lime для регрессии и таблиц, который с помощью линейной регрессии и заданного ядра строит интерпретацию. В качестве сложности используйте способ с регуляризацией (можете LARS). При вызове он должен вернуть коэффициенты регрессии, использованные для интерпретации. Не забудьте отшкалиировать фичи. В качестве метрики используйте евклидову.
Сделайте упрощенное семплирование - только нормальное распределение и дискретное по частотности категорий.

In [ ]:
class LIME:
    def __init__(self, train_df, kernel, kernel_width, categorical_features=None):
        pass

    def _generate_sample(self, instance):
        pass
        
    def explain(self, instance, prediction, num_features=5):
        pass
        